# CellOracle 개요 ― “GRN + in silico perturbation”
CellOracle는 단일-세포 전사체(scRNA-seq)나 scATAC-seq로부터 **세포-특이적 유전자 조절 네트워크(GRN)**를 추정한 뒤,
해당 GRN을 기반으로 전사인자(TF)·유전자 조작(KO/OE/점변이 등)을 컴퓨터 안에서 시뮬레이션해 세포 상태 변화를 예측하는 Python 라이브러리입니다.
구현된 핵심 모듈은 Oracle 클래스(데이터·모델·시뮬레이션 담당)와 Links 클래스(GRN 엣지 저장/분석) 두 개로 간단합니다.
morris-lab.github.io

1. 주요 산출물(Outputs)
산출물	형식	의미/용도
Oracle 객체	co.Oracle 인스턴스 (oracle.hdf5로 저장 가능)	전 과정의 메타 컨테이너. 원본 AnnData, 추정된 GRN, 시뮬레이션 결과, 시각화 좌표 등을 가진다.
Links 객체	co.Links 인스턴스 (links.hdf5)	(cluster × TF × target) 가중치 테이블. GRN 엣지 가중치와 분석 메서드 내장.
Imputed expression layer	oracle.adata.layers["imputed_count"]	GRN 기반 KNN-보간(count 복원) 결과.
Perturbation prediction	oracle.simulation_result dict	{ perturb_name: AnnData } 구조. 각 KO/OE 후 세포별 예상 발현·UMAP 벡터 필드 포함.
Vector-field / Stream plot	matplotlib figure	시뮬레이션-예측 세포 이동 경로.
Regulon importance scores	oracle.network_score	TF·gene 레벨의 중심성/필요도 지표.

In [ ]:
# Cell oracle dependency 해결

In [ ]:
conda create -n celloracle python=3.10
conda install -c conda-forge \
  numpy=1.26.4 \
  pandas=1.5.3 \
  anndata=0.10.8 \
  pybind11>=2.12 \
  pip \
  ipykernel \
  jupyterlab
conda activate celloracle
pip install --no-deps celloracle==0.20.0
conda install -c conda-forge numpy=1.26.4 pandas=1.5.3 anndata=0.10.8 pybind11>=2.12 pip ipykernel jupyterlab
conda install -c anndata=0.10.8 pybind11>=2.12

# Initiation

In [11]:
import scanpy as sc, celloracle as co, pandas as pd, scipy
############################################
# 0) 데이터 로드 & 전처리
############################################
adata = sc.read_h5ad("/data/kjc1/projects/#130.stroke/sobj/h5ad/is.h5ad")        # 이미 HVG 2-3 k 선택된 상태 권장
adata.layers["raw_count"] = adata.X.copy() # 원본 count 백업
# (필요 시 adata.obsm["X_umap"] 생성)
cluster_key="integrated_1.1"
root_dir="/data/kjc1/projects/#130.stroke/sobj/h5ad/"

In [12]:
adata_orig=adata

# oracle expect 1k~3k genes. over 3k genes can make the model unreliable, under 1k(effectively; by flavor, top_genes can be less than your input) can make subscript out of bounds error
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=1200,
    flavor="seurat_v3",# 추천 옵션 중 하나
    subset=True # 이 옵션을 켜면 자동으로 adata = adata[:, var.highly_variable] 해줌
)
# 늘어난 후보 중 최상위 1200개 뽑고, 그중 상위 1000개가 score 계산에 쓰이므로 안전

adata.raw=adata.copy()
adata

AnnData object with n_obs × n_vars = 33878 × 1200
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'Barcode', 'Best_Sample', 'Best_Probability', 'Second_Best_Sample', 'Second_Best_Probability', 'Probability_Ratio', 'droplet_demulti', 'GEM', 'join_key', 'day', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.0.8', 'seurat_clusters', 'droplet', 'sample_hto', 'GEM_hto', 'ol_time_point', 'sample_no', 'hos_no', 'sample_type', 'experiment_number', 'name', 'sex', 'birthdate', 'age', 'type', 'adm_date', 'sx_onset', 'blood_sample_time', 'csf_yn', 'csf_sample_time', 'csf_from', 'sample_done_time', 'exp_sample_no', 'set', 'multi_method', 'hto_no', 'charge', 'crcs_no', 'hash_id', 'sample_no.1', 'male', 'ht', 'wt', 'last_normal_dt', 'symptom_onset_dt', 'arrival_dt', 'blood_sample_time.1', 'LNT_to_sample_h', 'FAT_to_sample_h', 'hx_pre_mrs', 'toast', 'ivt', 'iat', 'ia_angio', 'pre_evt_state', 'post_evt_state', 'succ_recan', 'iv_start', 'ia_start', 'ia_end', 'lnt_to_iat_h', 'iat_to_sample_h', 'sHT', 'HT

In [ ]:
# ncomp determination
from sklearn.decomposition import PCA
import numpy as np

# adata.X 또는 adata.layers["raw_count"]에 PCA 적용
pca = PCA(n_components=min(adata.n_obs, adata.n_vars))
pca.fit(adata.X.toarray() if scipy.sparse.issparse(adata.X) else adata.X)
cumvar = np.cumsum(pca.explained_variance_ratio_)

# 예: 80% 누적 분산 지점 찾기
n_comps = int(np.searchsorted(cumvar, 0.80) + 1)
print(f"80% 분산 누적 지점의 PC 개수: {n_comps}")


In [ ]:
#k decision
# 전체 세포 수의 1%–5% 범위를 권장
pct = 0.03   # 3%
k = max(10, int(pct * adata.n_obs))
print(f"k (이웃 수): {k}")

In [21]:
n_comps=20
k=500
# k=800
# k=1000

In [22]:



############################################
# 1) Oracle 객체 생성 & 데이터 주입
############################################
oracle = co.Oracle()
oracle.import_anndata_as_raw_count(
        adata              = adata,
        cluster_column_name= cluster_key,     # obs 내 클러스터 컬럼명
        embedding_name     = "X_umap"       # obsm 키
)

############################################
# 2) Base-GRN 불러오기 (예: 마우스 sci-ATAC atlas)
############################################
# base_grn = co.data.load_mouse_scATAC_atlas_base_GRN()   # 내 ATAC-seq 결과로 직접 만들 수도 있음
# 인간 데이터라면
base_grn = co.data.load_human_promoter_base_GRN()
oracle.import_TF_data(TF_info_matrix = base_grn)

############################################
# 3) KNN imputation + GRN 추정
############################################
oracle.perform_PCA()
# → 중요 PC 개수(n_comps) 결정 (예: 30)
oracle.knn_imputation(
    n_pca_dims = 20,
    k          = int(0.03 * oracle.adata.shape[0]),  # 예: 세포 수의 3%
    balanced   = True,
    b_sight    = k*8,
    b_maxl     = k*4,
    n_jobs     = 32
)

# 1) 기존 fit_J / infer_grn / export_links 단계 대신…
links_obj = oracle.get_links(
    cluster_name_for_GRN_unit = cluster_key,   # 예: "louvain"
    alpha                     = 10,            # 규제 세기: 1~20 사이 실험 추천
    bagging_number            = 20,            # bagging 반복 횟수 (기본 20)
    verbose_level             = 1,             # 0:무출력, 1:프로그레스바, 2~:디테일
    model_method              = "bagging_ridge",# "bagging_ridge" 또는 "bayesian_ridge"
    n_jobs                    = 32             # 병렬 코어 수
)
links_obj.to_hdf5(root_dir+"links_obj.celloracle.links")
# 1) GRN 필터링 (예: 상위 10,000 엣지만)
links_obj.filter_links(p=0.001, weight="coef_abs", threshold_number=10000)

# 2) 시뮬레이션 준비
oracle.get_cluster_specific_TFdict_from_Links(links_obj)
oracle.fit_GRN_for_simulation(alpha=10, verbose_level=1)

# 3) perturbation & 시각화
oracle.simulate_shift({"GATA1": 0.0}, n_propagation=3)
oracle.visualize_simulation_stream(
    color_by=cluster_key,
    embedding_name="X_umap",
    basis="prediction"
)
oracle.to_hdf5(root_dir+"oracle_Gata1_KO.celloracle.oracle")


Loading prebuilt promoter base-GRN. Version: hg19_gimmemotifsv5_fpr2
Total number of TF was 44. Although we can go to the GRN calculation with this data, but the TF number is small.


AttributeError: 'Oracle' object has no attribute 'fit_J'

In [ ]:
adata=adata_orig

In [ ]:



############################################
# 1) Oracle 객체 생성 & 데이터 주입
############################################
oracle = co.Oracle()
oracle.import_anndata_as_raw_count(
        adata              = adata,
        cluster_column_name= cluster_key,     # obs 내 클러스터 컬럼명
        embedding_name     = "X_umap"       # obsm 키
)

############################################
# 2) Base-GRN 불러오기 (예: 마우스 sci-ATAC atlas)
############################################
# base_grn = co.data.load_mouse_scATAC_atlas_base_GRN()   # 내 ATAC-seq 결과로 직접 만들 수도 있음
# 인간 데이터라면
base_grn = co.data.load_human_promoter_base_GRN()
oracle.import_TF_data(TF_info_matrix = base_grn)

############################################
# 3) KNN imputation + GRN 추정
############################################
oracle.perform_PCA()
# → 중요 PC 개수(n_comps) 결정 (예: 30)
oracle.knn_imputation(
    n_pca_dims = 20,
    k          = int(0.03 * oracle.adata.shape[0]),  # 예: 세포 수의 3%
    balanced   = True,
    b_sight    = k*8,
    b_maxl     = k*4,
    n_jobs     = 32
)

# 1) 기존 fit_J / infer_grn / export_links 단계 대신…
links_obj = oracle.get_links(
    cluster_name_for_GRN_unit = cluster_key,   # 예: "louvain"
    alpha                     = 10,            # 규제 세기: 1~20 사이 실험 추천
    bagging_number            = 20,            # bagging 반복 횟수 (기본 20)
    verbose_level             = 1,             # 0:무출력, 1:프로그레스바, 2~:디테일
    model_method              = "bagging_ridge",# "bagging_ridge" 또는 "bayesian_ridge"
    n_jobs                    = 32             # 병렬 코어 수
)
links_obj.to_hdf5(root_dir+"links2.h5")
# 1) GRN 필터링 (예: 상위 10,000 엣지만)
links_obj.filter_links(p=0.001, weight="coef_abs", threshold_number=10000)

# 2) 시뮬레이션 준비
oracle.get_cluster_specific_TFdict_from_Links(links_obj)
oracle.fit_GRN_for_simulation(alpha=10, verbose_level=1)

# 3) perturbation & 시각화
oracle.simulate_shift({"GATA1": 0.0}, n_propagation=3)
oracle.visualize_simulation_stream(
    color_by=cluster_key,
    embedding_name="X_umap",
    basis="prediction"
)
oracle.save_oracle(root_dir+"oracle_Gata1_KO2.hdf5")


# 4. 중요 함수별 설명 & 실전 팁
단계	핵심 함수	파라미터 팁	해석/주의
KNN Imputation	get_knn_imputation()	n_neighbors=cell × 0.01 ~ 0.05	주변 유사 세포 기반 결측/희소 count 보정.
Motif 스캔(J)	fit_J(window=1000)	프롬프터 기반 프로모터 길이 조정	window 길이 ↑ → False-positive 엣지 ↑
GRN 추정(W)	infer_grn(alpha=0.05)	alpha = Lasso 규제 강도	과규제 시 엣지 소실, 저규제 시 과밀
Links 필터링	links.filter_links()	min_coefficient, min_abs_corr	스파스한 GRN 선호→Downstream 속도 개선
Simulation fit	fit_GRN_for_simulation(alpha=10)	Ridge α 값	예측 안정성↑ vs 세부감도↓, α=5-20 탐색
Perturbation	simulate_shift({gene: val})	- KO: 0, - OE: 2 × max expr	드라마틱 OE 값은 벡터 필드 왜곡 위험
시각화	visualize_simulation_stream()	basis="prediction"/"difference"	실제 UMAP → 방향 벡터 레이어로 오버레이

# 5. CellOracle 결과를 해석하는 법 (과외 Q&A 스타일)
Q1. Links 객체에서 coefficient 값이 클수록 무슨 의미인가요?
A. LASSO/Ridge에서 나온 **가중치(W)**로, TF → target gene의 양·음 영향도 및 크기를 나타냅니다. 양수 = 활성, 음수 = 억제 경향으로 해석합니다. 절대값이 0에 가까우면 ‘근거 희박’.

Q2. simulation_result에서 각 세포의 dX, velocity 벡터는?
A. GRN을 선형 ODE( dX = W·X )로 근사하여 n_propagation 스텝 동안 퍼뜨린 후,
시작 세포-표현형(UMAP 좌표) 대비 이동량을 벡터로 기록한 것입니다.
0 벡터 : 해당 세포 상태가 perturbation의 평형점에 이미 가까움.

Q3. 여러 TF를 동시에 KO/OE하려면?

python
oracle.simulate_shift(
       perturb_condition={"Gata1":0.0, "Tal1":0.0},
       n_propagation=3)
딕셔너리 key-value로 몇 개든 지정할 수 있습니다.

Q4. GRN 품질을 빠르게 평가하는 지표가 있나요?
oracle.network_score (Shannon entropy 기반)이나
links_obj.plot_degree_distribution()으로 엣지 분포를 확인해 과-희소/과-조밀 여부를 시각적으로 판단합니다.

# 6. 추가 읽을거리
Nature (2024) 논문: CellOracle로 84개 TF KO 시뮬레이션을 통해 혈액 분화 경로를 검증.
Nature

Mini-review (2024) : 단일-세포 perturbation 모델링 도구 비교 (CellOracle vs scGen 등).
PMC

공식 문서 & 튜토리얼: step-by-step Jupyter 노트북 제공.
morris-lab.github.io

# 마무리 조언
**데이터 준비가 80 %**입니다. 엄격한 HVG 필터와 batch correction(Scanpy bbknn/scVI) 후 CellOracle에 투입하세요.

scATAC 기반 base-GRN을 쓰면 예측 정확도가 확연히 향상됩니다.

Perturbation 결과는 가설 생성 용도입니다. Wet lab validation을 꼭 병행하세요!